# Notebook modelo para o treinamento de modelos
Este notebook jupyter é um modelo de exemplo para o treinamento de modelos. Ele está organizado para ser simples de entender e de fácil modificação, conforme a necessidade.

### 1.Importação de bibliotecas

In [2]:
import os, sys
import numpy as np

# Adiciona a pasta TrafficSoundAnalysis para o 'PATH', para que o python reconheça nosso pacote. Se isso falhar, não será possível fazer a importação dos pacotes
sys.path.append(os.path.dirname(os.path.abspath('')))

# Importa a classe DataHandler e ModelHandler da nossa biblioteca de tráfego
from TrafficSoundAnalysis.DataHandler.DataHandler import *
from TrafficSoundAnalysis.ModelHandler.ModelHandler import *

# Importa o Tensorflow
import tensorflow as tf
# Força o Tensorflow a usar apenas uma GPU. Como a FEBE é compartilhada entre muitos usuários, isso é importante.
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

"""
# Isso aqui é para resolve um problema de 'Out Of Memory' na minha gpu (GTX 1650, 4gb)
# Comentado aqui por que acho que não é necessário na FEBE.
# https://stackoverflow.com/questions/59873568/unknownerror-failed-to-get-convolution-algorithm

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)
"""

# Carrega o dicionário python com todos os folds
from folds import *

# Tensorboard
%load_ext tensorboard

### 2.Importação do dataset extraído

In [3]:
# 'path' para os dados. No caso, eles ainda estão na minha pasta principal.
dataset_dir = '/home/mathlima/dataset'
if not os.path.isdir(dataset_dir):
    raise Exception("Pasta de dataset não encontrada")

# inicialização do objeto "DataHandler", que gerenciará os dados.
dataHandler = DataHandler(dataset_directory = dataset_dir, image_format = [224, 224, 3])

### 3.Geração do modelo a ser treinado

In [4]:
#   Cria um objeto do tipo ModelHandler() para geração do nosso modelo
model = ModelHandler()

Inicialização de modelo por dicionário python

In [3]:
#   Modelo de dicionario python para definicao de uma rede
network = dict()

#   Model and training configuration
network['model_name'] = 'training_example'
network['input_format'] = [224, 224, 3]

#   Convolutional layer
network['cnn'] = 'vgg16' #or inceptionv3, or resnet50
network['cnn_offline'] = True
network['cnn_freeze_imagenet_weights'] = True

#   Pooling layer
network['pooling'] = 'gap'  #or 'none', or 'gmp'

#   RNN Layer
network['rnn'] = 'lstm' #   or 'lstm'
network['rnn_timesteps'] = 32
network['rnn_outputsize'] = 128
network['rnn_dropout'] = 0.2
network['rnn_isstateful'] = False

#   Hidden FC layer
network['hiddenfc'] = True
network['hiddenfc_size'] = 128
network['hiddenfc_activation'] = 'tanh'
network['hiddenfc_regularizer'] = None
network['hiddenfc_dropout'] = 0.2

#   Dataset related things
network['dataset_overlapwindows'] = True
network['dataset_causalprediction'] = False

In [6]:
model.LoadModelFromDictionary(network)

# O modelo keras gerado fica salvo no parametro '.model' do nosso objeto.
# Vamos sumarizar o modelo gerado:

# Summary
model.model.summary()

# Também podemos vizualizar o modelo com o método ShowModel()
from IPython.display import Image

model.ShowModel()
Image(filename='model.png')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                32832     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 14,747,585
Trainable params: 32,897
Non-trainable params: 14,714,688
__________________________________________

### 4.Treinamento de redes neurais

In [12]:
#   Carregamos os dados necessários (Nesse caso, os dados do fold 0)
[
    x_train,
    y_train,
    x_val,
    y_val
] = dataHandler.LoadDataset(folds[0],
                            CNN="vgg16",
                            CNN_offline=True,
                            Pooling="gap",
                            LSTM=True,
                            time_steps=32,
                            overlap_windows=True,
                            causal_prediction=False)

print('x_train.shape:',x_train.shape)
print('y_train.shape:',y_train.shape)
print('x_val.shape:',x_val.shape)
print('y_val.shape:',y_val.shape)

[INFO]: Loading non-extracted data from fold fold_0
[INFO]: Loading data
[INFO]: Starting building process for train dataset
[INFO]: Size of ndarray of type np.uint8: 6.66796875 mb
[INFO]: Size of ndarray of type np.ufloat32: 26.671875 mb
[INFO]: Starting building process for test dataset
[INFO]: Fold built is complete
[INFO]: Dataset loaded sucessefully
x_train.shape: (569, 64, 64, 3)
y_train.shape: (569,)
x_val.shape: (252, 64, 64, 3)
y_val.shape: (252,)


__Criação de 'Callbacks'__

In [ ]:
# Learning Rate Callback
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.0001,
    decay_steps=10,
    decay_rate=0.5,
    staircase=False)

learning_schedule = keras.callbacks.LearningRateScheduler(lr_schedule)

# Early stop Callback
earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# ModelCheckpoint Callback
model_checkpoint = keras.callbacks.ModelCheckpoint(
                    'model_checkpoint.hdf5',
                    monitor='val_loss',
                    verbose=2,
                    save_best_only=True,
                    save_weights_only=False,
                    mode='auto')

# Tensorboard
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='logs')

# Geração dos callbacks para passar para o metodo .fit
callback = [learning_schedule, model_checkpoint, earlystop, tensorboard]

__model.fit__

In [13]:
# Seguindo o padrão do Keras, primeiro compilamos o modelo:
model.CompileModel()

# E em seguida o treinamos com:
fit_history = model.Train(x=x_train, y=y_train, validation_data=(x_val, y_val), epochs=10, callbacks=callback)

# Por fim, salvamos o modelo no disco, da seguinte forma:
model.SaveModel('.', fit_history=fit_history)

[INFO]: Model was defined in the following dictionary:
	model_name = training_example
	input_format = [64, 64, 3]
	cnn = vgg16
	cnn_offline = False
	cnn_freeze_imagenet_weights = True
	pooling = gap
	rnn = none
	rnn_timesteps = 10
	rnn_outputsize = 64
	rnn_dropout = 0.2
	rnn_isstateful = False
	hiddenfc = True
	hiddenfc_size = 64
	hiddenfc_activation = tanh
	hiddenfc_regularizer = None
	hiddenfc_dropout = 0
	dataset_overlapwindows = True
	dataset_causalprediction = False
[INFO]: Model 'training_example' summary:
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 64, 64, 3)]       0         
_________________________________________________________________
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 512)               0         
___